In [4]:
import numpy as np
import pandas as pd

In [6]:
features = pd.read_csv("../data/features/user_behaviour.csv")
features.head()

,Unnamed: 0,profile_id,age,income,gender_F,gender_M,gender_O,become_member_on_date,days_since_last_member,channel_email,...,reward_y,difficulty,channel_mobile,duration,channel_social,offer_type_informational,offer_type_discount,time,reward_x,offer_type_bogo
0,0,0009655768c64bdeb2e877511632db8f,33,72000.0,False,True,False,2017-04-21,461,9,...,16.0,40.0,9,51.0,6,4,3,9414,9.0,2
1,1,0011e0d4e6b944f998e987f904e8c1e5,40,57000.0,False,False,True,2018-01-09,198,10,...,26.0,64.0,8,62.0,4,4,4,6570,13.0,2
2,2,0020c2b971eb4e9188eac86d93036a77,59,90000.0,True,False,False,2016-03-04,874,8,...,36.0,60.0,8,53.0,8,2,3,6378,14.0,3
3,3,0020ccbbb6d84e358d3414a3ff76cffd,24,60000.0,True,False,False,2016-11-11,622,8,...,26.0,34.0,8,44.0,6,2,2,8622,13.0,4
4,4,003d66b6608740288d6cc97a6903f4f0,26,73000.0,True,False,False,2017-06-21,400,9,...,13.0,60.0,8,64.0,6,4,5,11718,9.0,0
